In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import scipy
from PIL import Image 
from keras.datasets import mnist
from keras import layers
from keras.layers import Dropout
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import initializers
from keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.layers.normalization import BatchNormalization
from array import array
import cv2
import os
import random
from google.colab.patches import cv2_imshow

Preprocessing and Data Augmentation

In [ ]:
new_shape = (128,128)#reshaping of the images as the images were too large to give out any output
def load_images_from_folder(folder,images,y,i):
  
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))#stacking of the images in an array for train test split
        
        #img = Image.open(os.path.join(folder,filename))
        #print(type(img))
        if img is not None:
            #print(img.shape)
            #plt.imshow(img)
            img = cv2.resize(img,new_shape)
            
            
            im=img.copy() #adding the gaussian noise
            mean=0
            st=0.7
            gauss = np.random.normal(mean,st,im.shape)
            gauss = gauss.astype('uint8')
            im = cv2.add(im,gauss)
            

            ima=img.copy() #adding the contrast 
            value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
            hsv = cv2.cvtColor(ima, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if value >= 0:
              lim = 255 - value
              v[v > lim] = 255
              v[v <= lim] += value
            else:
              lim = np.absolute(value)
              v[v < lim] = 0
              v[v >= lim] -= np.absolute(value)

            final_hsv = cv2.merge((h, s, v))
            ima = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
          
            im1=img.copy()#adding the saturation
            value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
            hsv = cv2.cvtColor(im1, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if value >= 0:
              lim = 255 - value
              s[s > lim] = 255
              s[s <= lim] += value
            else:
              lim = np.absolute(value)
              s[s < lim] = 0
              s[s >= lim] -= np.absolute(value)

            final_hsv = cv2.merge((h, s, v))
            im1 = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

            im2=img.copy()#altering the brightness
            brightness = 10
            contrast = random.randint(40, 100)
            dummy = np.int16(im2)
            dummy = dummy * (contrast/127+1) - contrast + brightness
            dummy = np.clip(dummy, 0, 255)
            im2 = np.uint8(dummy)

            images.append(img)
            images.append(im)
            images.append(ima)
            images.append(im1)
            images.append(im2)
            
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            #i=i+1
    #print(i)
    return images,y

X = []
y=[]
for i in range(1,63):#fetching the images from folder
  #print(i)
  if (i<10):
    folder="/content/gdrive/MyDrive/trainPart1/train/Sample00"+str(i)
  else:
    folder="/content/gdrive/MyDrive/trainPart1/train/Sample0"+str(i)
  X,y=load_images_from_folder(folder,X,y,i)
  


In [ ]:
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5
print(train_images.shape)
# Flatten the images.


#train_images = train_images.reshape((-1, 784,3))
print(train_images.shape)
#test_images = test_images.reshape((-1, 784,3))
#val_images = val_images.reshape((-1, 784,3))


(10540, 128, 128, 3)
(10540, 128, 128, 3)


In [ ]:

# # Build the model,inital model to just experiment
# model = Sequential() # create model
# model.add(Dense(64, input_dim=784, trainable=True,activation='relu', use_bias=True, 
#                 kernel_initializer=initializers.he_normal(seed=None))) # hidden 
# model.add(Dropout(0.5))


# model.add(Dense(64, use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# model.add(Dense(62, trainable=True, activation='softmax')) # output layer

model = Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=62, activation = 'softmax'))

This is training block where hyperparameter tuning is required

Testing


In [ ]:

scores = model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])

55/55 [==============================] - 0s 6ms/step - loss: 0.1777 - accuracy: 0.9403
Test accuracy: 0.9403409361839294
